# Prerequisites

## Software package needed for Spotipy

In [1]:
#installing spotipy to perform queries against Spotify
!pip install spotipy --upgrade
!pip install fsspec
!pip install python-Levenshtein

     |████████████████████████████████| 92kB 3.7MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144803 sha256=9d927820cf5cce9b7e5c292cafd8bbce6966157015f5c26bb1624b83f23616ce
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Python Libraries used

In [4]:
import pandas as pd
import numpy as np
import time
import math

from difflib import SequenceMatcher #used for comparing similarity of strings
import Levenshtein 

import requests
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials('41f67e00313649c0962534596da52a38','2c885133946d4253b92f50e19bcd9308')
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:

#this reads in the Kaggle data from the Shared Google Drive for the project
df = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/data.csv')

#print(df.loc[df['name']=="Boom Boom Pow"]['popularity'])

# Reusable Functions 

## getFromSpotify

This function is used for retrieving songs from Spotify by year

In [ ]:
def getFromSpotify(year=2020):
  sp_list = []

  #extract songs by release year
  for count in range(40):
    try:
      sInfo = sp.search(q='year:"'+str(year)+'"',
                        type='track',
                        limit=50,offset=50*count)
      for a in sInfo['tracks']['items']:
        dict1={}
        dict1['name'] = a['name']
        dict1['artist_name'] = a['artists'][0]['name']
        dict1['track_id'] = a['id']
        dict1['track_number'] = a['track_number']
        dict1['popularity'] = a['popularity']
        dict1['release_date'] = a['album']['release_date']
        sp_list.append(dict1)
    except Exception as e:
      break
  
  return sp_list

In [ ]:
df1=pd.DataFrame(getFromSpotify(2020))

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          2000 non-null   object
 1   artist_name   2000 non-null   object
 2   track_id      2000 non-null   object
 3   track_number  2000 non-null   int64 
 4   popularity    2000 non-null   int64 
 5   release_date  2000 non-null   object
dtypes: int64(2), object(4)
memory usage: 93.9+ KB


# Billboard Data and Spotify

## Load Billboard Data from previous web scraping

In [ ]:
#load billboard dataset extracted by web scraping code. 
df_bb = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/billboardSongs2009_2019_5.csv')

df_bb=df_bb.drop(df_bb.columns[0], axis=1) #drop unecessary column picked up from the index column
df_bb=df_bb[df_bb['year']>=2009]
df_bb=df_bb.drop_duplicates(['title', 'artist'])
df_bb.reset_index(inplace=True, drop=True)
df_bb.info

<bound method DataFrame.info of      number            title                             artist  year
0         1    Boom Boom Pow                The Black Eyed Peas  2009
1         2       Poker Face                          Lady Gaga  2009
2         3       Just Dance  Lady Gaga featuring Colby O'Donis  2009
3         4  I Gotta Feeling                The Black Eyed Peas  2009
4         5       Love Story                       Taylor Swift  2009
..      ...              ...                                ...   ...
986      96      Eyes on You                         Chase Rice  2019
987      97    All to Myself                         Dan + Shay  2019
988      98        Boyfriend     Ariana Grande and Social House  2019
989      99     Walk Me Home                               P!nk  2019
990     100          Robbery                         Juice Wrld  2019

[991 rows x 4 columns]>

## Use Spotipy's search function

Using Spotipy's search function that can return up to 2000 results, (50 in one search)

In [ ]:
# def getSongIDfromSpotify(title="",artist="",year=2000):
#   sInfo = sp.search(q='track:"' + title + '"',limit=50,type='track') # + '" artist:"' + artist + '"' + 'year:' + str(year-1) +"-" +str(year+1) ' artist:'+artist
  
#   # #if results are close to 50, it means the song name search is too generic!
#   # if(len(sInfo['tracks']['items'])>49):
#   #   sInfo = sp.search(q='track:"' + title + '" artist:"' + artist + '"',limit=50,type='track')  
#   #   print(len(sInfo['tracks']['items']), title, artist)
#   rows_list = []
#   df_song = dict()
#   df_song['track_id']=np.NaN
#   df_song['popularity']=np.NaN
#   df_song['artist_name']=np.NaN

#   for idx, song in enumerate(sInfo['tracks']['items']):
#     dict1={}
#     dict1['track_id'] = song['id']
#     dict1['popularity'] = int(song['popularity'])
#     #try:
#     if len(song['artists'][0])>0:
#       dict1['artist_name']=song['artists'][0]['name']
      
#     rows_list.append(dict1)
#     df_song = pd.DataFrame(rows_list)
#     df_song=df_song.sort_values("popularity", ascending=False).head(1)
  
#   return df_song

In [5]:
#2nd version dev
def getSongIDfromSpotify(title,artist,year):
  rows_list = []
  df_song = dict()
  df_song['track_id']=np.NaN
  df_song['popularity']=np.NaN
  df_song['artist_name']=np.NaN
  df_search = pd.DataFrame()

  for count in range(1):
    sInfo = sp.search(q='track:' + title, limit=50, offset=count*50, type='track')
    if len(sInfo['tracks']['items'])>0:
      df_search = df_search.append(pd.json_normalize(sInfo['tracks']['items'])) # 
    else: break

  if len(sInfo['tracks']['items'])>0:
    #df_search = pd.json_normalize(sInfo['tracks']['items'])
    #print(title)
    df_search.sort_values('popularity',ascending=False,inplace=True)

    for idx, a in enumerate(df_search.artists):
      if artist.find(a[0]['name'].lower()) > -1:
        dict1={}
        #print(idx, title, artist, a[0]['name'], df_search.iloc[idx]['id'], df_search.iloc[idx]['popularity'])
        dict1['track_id'] = df_search.iloc[idx]['id']
        dict1['popularity'] = df_search.iloc[idx]['popularity']
        dict1['artist_name'] = a[0]['name']
        rows_list.append(dict1)
        df_song = pd.DataFrame(rows_list)
        df_song=df_song.head(1)
        break
  
  return df_song

In [6]:
df_bb_ids=pd.DataFrame()
df_no_bb = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/no_billboard.csv')

for index, row in df_bb.iterrows():
  song_name=row['title'] #.replace("'","\'")
  artist=row['artist']
  df_temp = getSongIDfromSpotify(song_name, artist.lower(), row['year'])
  df_bb_ids = df_bb_ids.append(df_temp, ignore_index=True, sort=False)

FileNotFoundError: ignored

## Read songs extracted from Spotify

In [ ]:
df_no_bb = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/no_billboard.csv')

In [ ]:
df_no_bb.head()

,name,artist_name,track_id,track_number,popularity,release_date
0,Bring It On Home - Digitally Remastered 00,Lou Rawls,7lemF5PY2O8aGkhHUyz2Bp,17,37,2000-06-06
1,Non-Zero Possibility,At The Drive In,7nuHLglk5FqpKY93cdJFIq,11,29,2000
2,Discipline 99 Pt. 0,Quasimoto,0X82TPGaYIcPpFA4LmKeD7,6,32,2000-06-13
3,F*** You,Ja Rule,6UJtMwc3fDSJwenXlp1b3D,8,42,2000-01-01
4,Catch Up (Featuring Infamous 2-0 & Fate Wilson),Ludacris,01uOzjaXPn3wZctfpxQTWX,13,29,2000-01-01


In [ ]:
print(df_bb_ids.info())
print(df_bb_ids.loc[df_bb_ids['track_id'].isnull()])
#print(sp.track('4a9xOJC6kgknv7cU4D99Ha'))

## Merge results with billboard data

In [ ]:
df_bb_res = pd.merge(df_bb, df_bb_ids, left_index=True, right_index=True, how='left')
df_bb_res.head(20)
df_bb_res.to_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs2009_2019_6.csv')

In [ ]:
df_bb_2 = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs2009_2019_2.csv')
df_bb_3 = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs2009_2019_3.csv')

In [ ]:
song= df_bb_2.iloc[29]['title'] #"DontTrustMe"
artist="" #df_bb_2.iloc[141]['artist']
print(song, artist)

result = sp.search(q='track:' + song ,limit=50, type='track',offset=0) #'" artist:"' + artist +
df_test = pd.json_normalize(result['tracks']['items'])

#print(len(result['tracks']['items']))

df_test.sort_values('popularity', ascending=False, inplace=True)

for idx, item in enumerate(df_test['name']):
  print(song, item, df_test.iloc[idx]['artists'][0]['name']) # item[0]['name'], item['popularity'])

#for idx, r in enumerate(result['tracks']['items']):
  #print(r['name'],r['artists'][0]['name'],r['popularity'])

Read features for manually retrieved IDs

In [ ]:
df_bb_miss = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs_withFeatures_v6-2.csv')
df_bb_miss.drop(df_bb_miss.columns[0], axis=1, inplace=True) #drop unecessary column picked up from the index column
df_bb_miss.drop('artist_name', axis=1, inplace=True) #drop unecessary column picked up from the index column

In [ ]:
import requests
import json 

df_bb_miss['artist_name'] = ""

for index, id in enumerate(df_bb_miss['track_id']):
  token = "BQBP4RzeeHbA3t5hp7vj8RqHBPLHwR04uUv-iZyK4EQBmJ6SYBQcEdx3aNRqhG7eDhuY5uZN-vl1lJ5l5N0k1-CoqLjl-1NCWXa6AC4aJ9VP955-Id1And_esOrF3qQIZTmbgo2on7fGe1LGa-vaIdhPstYFFalvaif9YUmb4u8r"
  site = "https://api.spotify.com/v1/tracks/" + id
  header = {'access_token': token}
  result = requests.get(site, header)
  data = pd.json_normalize(json.loads(result.content))
  #print(data['popularity'])
  df_bb_miss.at[index,'popularity'] = data['popularity']
  artists = pd.json_normalize(data['artists'][0])
  print(artists['name'][0])
  df_bb_miss.at[index,'artist_name'] = str(artists['name'][0])
  #df_bb_miss.at[index, 'artist_name'] = 
  #print(json.loads(data))

In [ ]:
df_bb_miss.to_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs_withFeatures_v6-2.csv')

## Saving Billboard data with Spotify info to CSV

In [ ]:
#df_bb_res.to_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs1946_2019wSpotify_3.csv')


In [ ]:
#Reload dataset for Billboard songs with Spotify Song Features
df_bb_res = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs1946_2019wSpotify_3.csv')

In [ ]:
df_bb_rem=df_bb_res.loc[df_bb_res['track_id'].isnull()]

for index, row in df_bb_rem.iterrows():
  artist = row['artist'].split(" featuring ")[0]
  result = getSongIDfromSpotify(row['title'],artist,row['year'])
  if str(result['track_id']) != "nan":
    df_bb_rem['track_id'][index] = result['track_id']
    df_bb_rem['popularity'][index] = result['popularity']

In [ ]:
df_bb_rem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364 entries, 14 to 6271
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  364 non-null    int64  
 1   number      364 non-null    object 
 2   title       364 non-null    object 
 3   artist      364 non-null    object 
 4   year        364 non-null    int64  
 5   track_id    105 non-null    object 
 6   popularity  105 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 32.8+ KB


In [ ]:
for index, row in df_bb_rem.iterrows():
  artist = row['artist'].split(" with ")[0]
  result = getSongIDfromSpotify(row['title'],artist,row['year'])
  if str(result['track_id']) != "nan":
    df_bb_rem['track_id'][index] = result['track_id']
    df_bb_rem['popularity'][index] = result['popularity']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_bb_rem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364 entries, 14 to 6271
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  364 non-null    int64  
 1   number      364 non-null    object 
 2   title       364 non-null    object 
 3   artist      364 non-null    object 
 4   year        364 non-null    int64  
 5   track_id    111 non-null    object 
 6   popularity  111 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 32.8+ KB


In [ ]:
for index, row in df_bb_rem.iterrows():
  if str(row['track_id']) == "nan":
    print(row['title'] + "::" + row['artist']) 

Re-load billboard data with Spotify

In [ ]:
features=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

import numpy as np

results_list=[]
for song_id in df_bb_miss['track_id']:
  dict1={}
  # if np.isnan(song_id):
  result = sp.audio_features(song_id)
  for f in features:
    dict1[f]=result[0][f]
  # else:
  #   for f in features:
  #     dict1[f]=np.NaN

  results_list.append(dict1)

In [ ]:
df_bb_2 = pd.DataFrame(results_list)
#df_bb.insert(len(df_bb.columns), features, results_list, True)

In [ ]:
data = pd.concat([df_bb_miss, df_bb_2], axis=1, sort=False )

In [ ]:
data.to_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs_withFeatures_v6-3.csv')

In [ ]:
data_bb=pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/bbSongs_withFeatures_v6.csv')
data_no_bb = pd.read_csv('/content/drive/My Drive/ML Group Project/Data Sets/no_bb_w_audio_feat.csv')


In [ ]:
data_no_bb.drop(data_no_bb.columns[0], axis=1, inplace=True)
data_no_bb.drop(['track_number'], axis=1, inplace=True)

In [ ]:
data_no_bb.info()
data_bb.info()

In [ ]:
data_no_bb.head(5)

,artist_name,track_id,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Lou Rawls,7lemF5PY2O8aGkhHUyz2Bp,37,2000-06-06,0.454,0.571,0,-8.977,1,0.0667,0.1420,0.000163,0.1790,0.792,184.125,audio_features,spotify:track:7lemF5PY2O8aGkhHUyz2Bp,https://api.spotify.com/v1/tracks/7lemF5PY2O8a...,https://api.spotify.com/v1/audio-analysis/7lem...,178627,4.0
1,At The Drive In,7nuHLglk5FqpKY93cdJFIq,29,2000,0.294,0.612,7,-6.579,1,0.0355,0.6060,0.113000,0.1040,0.187,74.374,audio_features,spotify:track:7nuHLglk5FqpKY93cdJFIq,https://api.spotify.com/v1/tracks/7nuHLglk5Fqp...,https://api.spotify.com/v1/audio-analysis/7nuH...,334213,4.0
2,Quasimoto,0X82TPGaYIcPpFA4LmKeD7,32,2000-06-13,0.675,0.623,1,-6.921,1,0.4090,0.0392,0.000000,0.8680,0.727,91.346,audio_features,spotify:track:0X82TPGaYIcPpFA4LmKeD7,https://api.spotify.com/v1/tracks/0X82TPGaYIcP...,https://api.spotify.com/v1/audio-analysis/0X82...,152680,4.0
3,Ja Rule,6UJtMwc3fDSJwenXlp1b3D,42,2000-01-01,0.812,0.849,11,-4.264,1,0.2600,0.0758,0.000000,0.0675,0.838,94.015,audio_features,spotify:track:6UJtMwc3fDSJwenXlp1b3D,https://api.spotify.com/v1/tracks/6UJtMwc3fDSJ...,https://api.spotify.com/v1/audio-analysis/6UJt...,253400,4.0
4,Ludacris,01uOzjaXPn3wZctfpxQTWX,29,2000-01-01,0.613,0.690,5,-6.575,1,0.1760,0.1530,0.000000,0.0816,0.247,77.995,audio_features,spotify:track:01uOzjaXPn3wZctfpxQTWX,https://api.spotify.com/v1/tracks/01uOzjaXPn3w...,https://api.spotify.com/v1/audio-analysis/01uO...,254347,4.0


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML Group Project/Data Sets/v2_with_genre.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787182 entries, 0 to 787181
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        787182 non-null  int64  
 1   acousticness      787182 non-null  float64
 2   analysis_url      787182 non-null  object 
 3   artist            787144 non-null  object 
 4   artist_name       787144 non-null  object 
 5   billboard         787182 non-null  int64  
 6   danceability      787181 non-null  float64
 7   duration_ms       787182 non-null  int64  
 8   energy            787182 non-null  float64
 9   instrumentalness  787182 non-null  float64
 10  key               787182 non-null  int64  
 11  liveness          787182 non-null  float64
 12  loudness          787182 non-null  float64
 13  mode              787182 non-null  int64  
 14  number            787182 non-null  int64  
 15  popularity        787182 non-null  int64  
 16  speechiness       78

In [ ]:
genres = pd.read_csv('/content/drive/MyDrive/ML Group Project/Data Sets/genre_by_artist.csv')


In [ ]:
list_genre = data['genres'].unique()
print(len(list_genre))
print(list_genre)

25060
['dance pop_pop_pop dance_pop rap' 'dance pop_pop_pop dance'
 'dance pop_pop_pop dance_post-teen pop' ...
 'anime score_japanese concert band' 'enka_j-pop_japanese city pop'
 'bow pop_japanese classical performance_violin']


In [ ]:
songs = pd.read_csv('/content/drive/MyDrive/ML Group Project/Data Sets/v2_with_genre.csv')

In [ ]:
songs_id = songs[['track_id', 'track_href']].copy()
songs_id[['release_date']] = np.NaN 
  


## Extract Release Date for songs from Spotify

In [ ]:
folder = '/content/drive/MyDrive/ML Group Project/Data Sets/'
bb_songs = 'bbSongs_withFeatures_v6.csv'
non_bb_songs = 'no_bb_w_audio_feat.csv'

In [ ]:
df_bb  = pd.read_csv(folder+bb_songs)
df_nbb = pd.read_csv(folder+non_bb_songs)

In [ ]:
data_main = pd.read_csv(folder+"v2_with_genre.csv")

In [ ]:
import requests
import json 

df_bb['release_date'] = ""

for index, site in enumerate(df_bb['track_href']):
  token = "BQAyQF2ioqRpzOnMEFbJvor_Cc8E-zxiltL17ZhNLSPajm53hkIMY-yGVij0lxnEGIakQVa6_7fmzMAamJHCxlurfWsY5QQ7ETDGlY5oqCDj4B7TKdsxkv7dHoXgU-lf-NrSHbrdl4vH-ubJ0WSKELtlVRA_mB5XYS3qTdRivdj8"
  header = {'access_token': token}
  #print(site)
  if df_bb.at[index, 'release_date']=="":
    time.sleep(0.2)
    result = requests.get(site, header)
    #print(result)
    data = pd.json_normalize(json.loads(result.content))
    try:
      #print(data['album.release_date'][0], df_bb.at[index,'track_id'])
      df_bb.at[index,'release_date'] = data['album.release_date'][0]
    except:
      df_bb.at[index,'release_date'] = None
  #df_bb_miss.at[index, 'artist_name'] = 
  #print(json.loads(data))


In [ ]:
import datetime

def is_date(string, format="%Y-%m-%d"):
    try: 
        datetime.datetime.strptime(string, format)
        return True

    except ValueError:
        return False

In [ ]:
format="%Y-%m-%d"

for y in df_bb['release_date']:
  if len(y)==4:
    print(y, is_date(y, format))


2009 False
2013 False
2009 False
2009 False
2008 False
2010 False
2009 False
2010 False
2010 False
2011 False
2011 False
2012 False
2011 False
2013 False
2012 False
2011 False
2012 False
2014 False
2012 False
2012 False
2015 False


In [ ]:
df_bb['release_year'] = df_bb['release_date'].str[:4]
df_bb['release_month'] = df_bb['release_date'].str[5:7]
df_bb['release_day'] =df_bb['release_date'].str[8:10]

In [ ]:
df_bb['release_year'] = df_bb['release_date'].str[:4]
df_bb['release_month'] = 0
df_bb['release_day'] = 0
df_bb['release_week'] = 0

for index, rd in enumerate(df_bb['release_date']):
  if is_date(rd):
    df_bb.at[index, 'release_month'] = rd[5:7]
    df_bb.at[index, 'release_day'] = rd[8:10]
    r_date = datetime.datetime.strptime(rd, format)
    df_bb.at[index, 'release_week'] = r_date.isocalendar()[1]

In [ ]:
df_bb_dates = df_bb[['track_id','release_date','release_year','release_month','release_day', 'release_week']].copy()

In [ ]:
print(df_bb_dates.loc[df_bb_dates['release_day'].isnull(),'track_id'])

Series([], Name: track_id, dtype: object)


In [ ]:
text = pd.json_normalize(json.loads(requests.get("https://api.spotify.com/v1/tracks/7JjPovtNiOZ27C9EOEopX8" ,header).content))
print(text['album.release_date'])

0    2018-07-25
Name: album.release_date, dtype: object


In [ ]:
df_nbb['release_year'] = df_nbb['release_date'].str[:4]
df_nbb['release_month'] = df_nbb['release_date'].str[5:7]
df_nbb['release_day'] =df_nbb['release_date'].str[8:10]

In [ ]:
df_nbb['release_year'] = df_nbb['release_date'].str[:4]
df_nbb['release_month'] = 0
df_nbb['release_day'] = 0
df_nbb['release_week'] = 0

for index, rd in enumerate(df_nbb['release_date']):
  if is_date(rd):
    df_nbb.at[index, 'release_month'] = rd[5:7]
    df_nbb.at[index, 'release_day'] = rd[8:10]
    r_date = datetime.datetime.strptime(rd, format)
    df_nbb.at[index, 'release_week'] = r_date.isocalendar()[1]

In [ ]:
df_nbb_dates = df_nbb[['track_id','release_date','release_year','release_month','release_day', 'release_week']].copy()

In [ ]:
print(df_nbb_dates.loc[df_nbb_dates['release_day'].isnull(),'release_date'])

Series([], Name: release_date, dtype: object)


In [ ]:
df_dates = df_bb_dates.append(df_nbb_dates)

In [ ]:
df_dates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204064 entries, 0 to 1203072
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   track_id       1204064 non-null  object
 1   release_date   1204064 non-null  object
 2   release_year   1204064 non-null  object
 3   release_month  1204064 non-null  int64 
 4   release_day    1204064 non-null  int64 
 5   release_week   1204064 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 64.3+ MB


In [ ]:
#print(data['track_id'])
df_dates_2 = df_dates[df_dates['track_id'].isin(data_main['track_id'].tolist())]

In [ ]:
df_dates_2.to_csv(folder+'v4_release_dates.csv')

In [ ]:
df_nbb_dates.info()
df_dates_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203073 entries, 0 to 1203072
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   track_id       1203073 non-null  object
 1   release_date   1203073 non-null  object
 2   release_year   1203073 non-null  object
 3   release_month  1203073 non-null  object
 4   release_day    1203073 non-null  object
dtypes: object(5)
memory usage: 45.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 787233 entries, 0 to 1203072
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   track_id       787233 non-null  object
 1   release_date   786948 non-null  object
 2   release_year   786948 non-null  object
 3   release_month  786948 non-null  object
 4   release_day    786948 non-null  object
dtypes: object(5)
memory usage: 36.0+ MB


In [ ]:
df_dates['release_date']=2000

NameError: ignored